# Task 3: Simulation of a GBS experiment

A template code for calculating a vibronic spectrum of $V_3$ using a sampling method, simulating a Gaussian Boson Sampling experiment.

Input Parameters.

Note: "Strawberry Fields" is a code created by Xanadu and is the driver behind GBS. This can be installed using **pip**, but see install instructions in the <a href="https://github.com/CDL-Quantum/CohortProject_2020/blob/master/Project_3_Franck_Condon_Factors/Project3_LandingPage.pdf"> Project 3 Landing Page</a>. Plotly is a nice plotting code, but is not necessary if you wish to use one of your choice.

In [8]:
from sys import argv
from strawberryfields.apps import vibronic, data, sample, plot
import numpy as np
from plotly import offline

Variables.

Note: The **inputfile** for $V_3$ is what you will have to create to run this script.

In [19]:
inputfile = open('Task2Code/input_for_exercise3.txt', "r") # The name of the input file for the last task

N=int(inputfile.readline())     #Number of Atoms
nmodes=3*N-6                    #Number of Modes

w=np.zeros(nmodes,float)        #vib. frequencies of ground electronic state (1/cm)
wp=np.zeros(nmodes,float)       #vib. frequencies of excited electronic state (1/cm)
Ud=np.zeros((nmodes,nmodes),float)   #Duschinsky Matrix (unitless)
delta=np.zeros(nmodes,float)    #Displacement Vector (unitless)

T = 500  # temperature

print(N)

3


Read in Parameters.

Note: This is a suggested format for reading the **inputfile** that you create. However, feel free to modify it for your convenience.

In [20]:
for i in range(nmodes):
    w[i]=float(inputfile.readline())
for i in range(nmodes):
    wp[i]=float(inputfile.readline())
for i in range(nmodes):
    for j in range(nmodes):
        Ud[i,j]=float(inputfile.readline())

for i in range(nmodes):
    delta[i]=float(inputfile.readline())

In [34]:
print(w)
print(wp)

[255.  255.  420.8]
[261.9 263.3 421.8]


In [21]:
print(delta)

[-2.05330e-04 -7.33908e-04 -1.08223e+00]


Use "Strawberry Fields" to calculate GBS Parameters based on the frequencies, Duschinsky matrix, and temperature that was provided as input.

In [22]:
t, U1, r, U2, alpha = vibronic.gbs_params(w, wp, Ud, delta, T)

Generate Samples and Calculate Energies of Samples.

Note: The number of samples may be changed if the spectrum is not converged.

In [32]:
#nr_samples = 20000
nr_samples = 10
s = sample.vibronic(t, U1, r, U2, alpha, nr_samples)
e = vibronic.energies(s, w, wp)

Plot the Spectrum

Note: You may choose your favourite plotting program if you wish.

In [35]:
spectrum = plot.spectrum(e, xmin=-300, xmax=2000)
offline.plot(spectrum, filename="spectrum_{}.html".format(nr_samples))

'spectrum_10.html'